In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import codecs, re, json ,math, unicodedata
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Lưu trữ và xử lý dữ liệu lớn/data/nga

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, Normalizer
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import logging
import json
import unicodedata
%matplotlib inline
import time
logging.basicConfig(level=logging.INFO)

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from joblib import dump, load
import random as rd
np.random.seed(40)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Lưu trữ và xử lý dữ liệu lớn/data/nga


### Data

In [35]:
address1 = json.load(open("../../address.txt","r", encoding="utf8"))
address = dict()
for district in address1.keys():
    address[unicodedata.normalize("NFD",district)]=dict()
    for key in address1[district].keys():
        address[unicodedata.normalize("NFD",district)][unicodedata.normalize("NFD",key)]=[unicodedata.normalize("NFD",item) for item in address1[district][key]]

address['thanh oai']['ward'].append(unicodedata.normalize("NFD","thụy khuê"))
address['hoàn kiếm']['street'].append(unicodedata.normalize("NFD","láng"))
address['đống đa']['street'].append(unicodedata.normalize("NFD","nguyễn huy tưởng"))
address['hà đông']['ward'].append(unicodedata.normalize("NFD","mỗ lao"))
address['hai bà trưng']['ward'].append(unicodedata.normalize("NFD","vân hồ 3"))

for district in address.keys():
    district = unicodedata.normalize("NFD",district)
    for key in address[district].keys():
        key = unicodedata.normalize("NFD",key)
        address[district][key] = {unicodedata.normalize("NFD",value) for value in address[district][key]}

In [36]:
data = pd.read_csv("../Clean1/resultFinall.csv", encoding='utf8')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229630 entries, 0 to 229629
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          229630 non-null  float64
 1   bedroom       229630 non-null  float64
 2   description   229630 non-null  object 
 3   direction     18870 non-null   object 
 4   district      229630 non-null  object 
 5   facade        108956 non-null  object 
 6   floor         229630 non-null  int64  
 7   price         229630 non-null  float64
 8   street        229104 non-null  object 
 9   time          229630 non-null  object 
 10  title         229629 non-null  object 
 11  toilet        229630 non-null  float64
 12  type          229630 non-null  object 
 13  ward          229630 non-null  object 
 14  web           229630 non-null  object 
 15  month         229630 non-null  int64  
 16  year          229630 non-null  int64  
 17  pricePerArea  229630 non-null  float64
 18  quar

In [38]:
data['title'].fillna("", inplace=True)
data['description'].fillna("", inplace=True)

data['title'] = data['title'].str.normalize("NFD")
data['description'] = data['description'].str.normalize("NFD")
data['street'] = data['street'].str.normalize("NFD")
data['ward'] = data['ward'].str.normalize("NFD")
data['district'] = data['district'].str.normalize("NFD")
data['type'] = data['type'].str.normalize("NFD")

data.drop(data[data['year']!=2020].index, inplace=True)
data.reset_index(drop=True, inplace=True)

In [39]:
data['district'].value_counts()

đống đa           28361
thanh xuân         28066
hoàng mai          23376
hà đông           20852
hai bà trưng      19224
cầu giấy        18765
long biên          13543
ba đình            11236
nam từ liêm      10978
tây hồ            9027
bắc từ liêm     6410
thanh trì           3275
hoài đức          1729
gia lâm             1694
hoàn kiếm         1160
đông anh             585
chương mỹ          120
đan phượng         119
sóc sơn             103
thanh oai              95
quốc oai             60
mê linh               60
thạch thất          51
sơn tây              43
thường tín         41
ba vì                  8
phú xuyên             5
phúc thọ              4
ứng hòa              2
Name: district, dtype: int64

In [40]:
data['district'].value_counts().keys()[:6]

data = data[data['district'].isin(['đống đa', 'thanh xuân', 'hoàng mai', 'cầu giấy', 'hà đông',
       'hai bà trưng'
    #    , 'ba đình', 'long biên', 'nam từ liêm',
    #    'tây hồ', 'bắc từ liêm'
       ])]

In [41]:
sum_wards = 0
for district in data['district'].unique():
    sum_wards += len(address[district]['ward'])
sum_wards
# co tong 156 phuong thuoc 6 quan

97

In [42]:
data2 = data.copy()

In [43]:
data = data2.copy()

In [44]:
data.drop(data[(data['pricePerArea']<10)].index, inplace=True)

In [45]:
data.drop(data[(data['pricePerArea']>1000) & (data['floor']<4)].index, inplace=True)

In [46]:
data.reset_index(drop=True, inplace=True)

In [47]:
data.groupby(['district','quarter']).size().unstack()

quarter,1,2,3,4
district,,,,
cầu giấy,397,585,17560,180
hai bà trưng,467,785,17762,199
hà đông,392,381,19905,130
hoàng mai,465,905,21694,279
thanh xuân,547,1054,26206,248
đống đa,735,1277,26018,310


In [48]:
data.groupby(['type','quarter','district']).size().unstack()

district                            cầu giấy  ...  đống đa
type                       quarter                ...           
biệt thự, liền kề  1                  10  ...         17
                           2                  31  ...         44
                           3                 729  ...        544
                           4                   3  ...         11
căn hộ, chung cư       1                 103  ...         54
                           2                 100  ...         99
                           3                2423  ...       1567
                           4                  37  ...         14
hẻm                       1                  42  ...        154
                           2                  71  ...        229
                           3                7372  ...      13384
                           4                  45  ...         92
nhà mặt phố           1                 219  ...        483
                           2                 357  ...        855
                           3                6378  ...       9710
                           4                  84  ...        183
đất                      1                  23  ...         27
                           2                  26  ...         50
                           3                 658  ...        813
                           4                  11  ...         10

[20 rows x 6 columns]

In [49]:
data.groupby("district")['pricePerArea'].describe()

,count,mean,std,min,25%,50%,75%,max
district,,,,,,,,
cầu giấy,18722.0,122.540989,65.789814,10.0,81.578947,110.000000,162.500000,1444.444444
hai bà trưng,19213.0,103.154124,61.666825,10.0,70.000000,90.000000,118.750000,2900.000000
hà đông,20808.0,76.702531,61.588311,10.0,40.625000,70.937500,100.000000,2550.000000
hoàng mai,23343.0,79.780871,51.028794,10.0,58.532999,75.581395,95.000000,2500.000000
thanh xuân,28055.0,99.698177,57.620173,10.0,73.000000,91.228070,116.129032,2450.000000
đống đa,28340.0,124.043636,70.364349,10.0,85.294118,107.142857,151.111111,2800.000000


In [50]:
data = data[['area','month','price','quarter','type','district','ward','floor','pricePerArea']]

In [51]:
data['type'].unique()

array(['biệt thự, liền kề ', 'nhà mặt phố',
       'căn hộ, chung cư', 'đất', 'hẻm'], dtype=object)

In [52]:
data.drop(data[data['type']=='đất'].index, inplace=True)
data.reset_index(drop=True, inplace=True)

In [53]:
data.drop(data[data['ward'].isnull()].index, inplace=True)
data.reset_index(drop=True, inplace=True)

In [54]:
wards_of_districts = []
for district in data['district'].unique():
    wards_of_districts.append(set(data['ward'][data['district']==district].unique()))

In [55]:
sum_wards = 0
unique_wards = set()
list_wards = []
for wards in wards_of_districts:
    print(len(wards))
    unique_wards.update(wards)
    sum_wards+=len(wards)
    list_wards.extend(list(wards))
print(sum_wards, len(unique_wards), list_wards)

for district in data['district'].unique():
    print(district, len(address[district]['ward']))

18
13
14
10
21
21
97 95 ['văn quán', 'quang trung', 'mộ lao', 'đồng mai', 'nguyễn trãi', 'phú lãm', 'dương nội', 'vạn phúc', 'kiến hưng', 'yết kiêu', 'phúc la', 'phú la', 'biên giang', 'phú lương', 'la khê', 'mỗ lao', 'yên nghĩa', 'hà cầu', 'nhân chính', 'thượng đình', 'khương hạ', 'phương mai', 'hạ đình', 'thanh xuân nam', 'phương liệt', 'thanh xuân trung', 'khương trung', 'kim giang', 'khương đình', 'khương mai', 'thanh xuân bắc', 'định công', 'trần phú', 'lĩnh nam', 'hoàng văn thụ', 'hoàng liệt', 'thanh trì', 'vĩnh hưng', 'thịnh liệt', 'mai động', 'giáp bát', 'đại kim', 'tân mai', 'yên sở', 'tương mai', 'nghĩa đô', 'dịch vọng', 'nghĩa tân', 'dịch vọng hậu', 'pháo đài láng', 'xuân thủy', 'mai dịch', 'quan hoa', 'yên hòa', 'trung hòa', 'láng thượng', 'nam đồng', 'văn miếu', 'trung phụng', 'kim liên', 'láng hạ', 'ô chợ dừa', 'cát linh', 'hàng bột', 'tr

In [56]:
df = pd.DataFrame({'ward':list_wards})
df.value_counts()

ward             
phương mai         2
quang trung          2
đống mác          1
mộ lao             1
la khê              1
                    ..
thanh xuân trung    1
thịnh liệt        1
thịnh quang         1
thổ quan           1
bách khoa           1
Length: 95, dtype: int64

In [57]:
list_ward_district = []
for district in data['district'].unique():
    for ward in address[district]['ward']:
        list_ward_district.append(ward.strip()+", "+district.strip()) # chuan hoa ddeer deex trong qua trinh test thuwr vi du
len(list_ward_district)#, list_ward_district

97

In [58]:
dict_type = dict(zip(list(data['type'].unique()), [i for i in range(1, len(data['type'].unique())+1)]))
dict_district = dict(zip(list(data['district'].unique()), [i for i in range(1, len(data['district'].unique())+1)]))
dict_ward = dict(zip(list_ward_district, [i for i in range(1, len(list_ward_district)+1)]))

In [59]:
data['ward, district'] = None
data['En_district'] = None

for idx in range(len(data)):
    try:
        data['En_district'][idx] = dict_district[data['district'][idx]]
        data['type'][idx] = dict_type[data['type'][idx]]
        data['ward, district'][idx] = dict_ward[data['ward'][idx].strip()+", "+data['district'][idx].strip()]
    except:
        print(data['ward'][idx].strip()+", "+data['district'][idx].strip())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
data1 = data.copy()

In [61]:
data = data1.copy()

In [62]:
data = data[(data['month']==9)]

In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96373 entries, 9 to 133523
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   area            96373 non-null  float64
 1   month           96373 non-null  int64  
 2   price           96373 non-null  float64
 3   quarter         96373 non-null  int64  
 4   type            96373 non-null  object 
 5   district        96373 non-null  object 
 6   ward            96373 non-null  object 
 7   floor           96373 non-null  int64  
 8   pricePerArea    96373 non-null  float64
 9   ward, district  96373 non-null  object 
 10  En_district     96373 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 8.8+ MB


### Linear Regression

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96373 entries, 9 to 133523
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   area            96373 non-null  float64
 1   month           96373 non-null  int64  
 2   price           96373 non-null  float64
 3   quarter         96373 non-null  int64  
 4   type            96373 non-null  object 
 5   district        96373 non-null  object 
 6   ward            96373 non-null  object 
 7   floor           96373 non-null  int64  
 8   pricePerArea    96373 non-null  float64
 9   ward, district  96373 non-null  object 
 10  En_district     96373 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 8.8+ MB


In [65]:
def split_train_test(Xy, test_size=0.2):
    list_data_train, list_data_test = list(), list()

    for i, (_, values) in enumerate(Xy.groupby(['type',"ward, district"])):
        n_rows = len(values)
        
        n_data_test = int(n_rows*test_size)
        if n_data_test > 0:
            row = np.zeros(n_rows, dtype='bool')
            row[np.random.choice(n_rows, size=n_data_test, replace=False).astype('int64')] = True
            list_data_test.append(values[row])
            list_data_train.append(values[np.logical_not(row)])

    data_train = pd.concat(list_data_train)
    data_test = pd.concat(list_data_test)
    return data_train, data_test

data_train, data_test = split_train_test(data[['type', 'area','En_district', 'ward, district', 'floor', 'pricePerArea']])

X_train, X_test, y_train, y_test = data_train.iloc[:, :-1], data_test.iloc[:, :-1], data_train.iloc[:, -1], data_test.iloc[:, -1]
X_train = pd.get_dummies(X_train, columns=['type','En_district','ward, district'])
X_test = pd.get_dummies(X_test, columns=['type','En_district','ward, district'])
X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values, y_test.values

print(X_train.shape, X_test.shape , y_train.shape, y_test.shape)
reg = LinearRegression() #  If True, the regressors X will be normalized before regression by subtracting the mean and dividing by the l2-norm.
reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

(77216, 108) (19114, 108) (77216,) (19114,)
Mean Absolute Error: 31.17356701175046
Mean Squared Error: 2564.7945448825826
Root Mean Squared Error: 50.64380065597943


In [68]:
pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})

,Actual,Predicted
0,75.714286,128.267120
1,143.750000,140.147461
2,128.571429,119.147614
3,125.000000,117.014252
4,191.666667,128.580856
...,...,...
19109,244.444444,127.203827
19110,97.058824,115.750168
19111,100.000000,115.750168
19112,113.636364,104.560028


In [66]:
dump(reg, '../../Code/reg.joblib')

['../../Code/reg.joblib']

In [67]:
reg = load('../../Code/reg.joblib')

In [69]:
def input_pred():
    house = np.ones(data_train.shape[1])
    type_house = int(input("""Loai bat dong san : \n
                            1. biệt thự, liền kề  \n
                            2. nhà mặt phố\n
                            3. căn hộ, chung cư\n
                            4. hẻm\n """))
    area = float(input("Dien tich(m2): "))
    floor = int(input("So tang: "))
    district = int(input("""Quận/huyện/thị xã:\n
                        1. hà đông\n
                        2. thanh xuân\n
                        3. hoàng mai\n
                        4. cầu giấy\n
                        5. đống đa\n
                        6. hai bà trưng\n"""))
    district_name = None
    for dis in dict_district.keys():
        if dict_district[dis] == district:
            district_name = dis
            break
    ward = unicodedata.normalize("NFD", input("Phường/xã/thị trấn: "+str(address[district_name]['ward'])+"\n").strip())

    ward_district = dict_ward[ward+", "+district_name]

    data_test1 = data_test.copy()
    df = pd.DataFrame([[type_house, area, district, ward_district, floor]], columns = ['type','area','En_district', 'ward, district', 'floor'])
    data_test_df = data_test1.iloc[:, :-1].append(df, ignore_index=True)

    en_data_test_df = pd.get_dummies(data_test_df, columns=['type','En_district','ward, district'])
    test = en_data_test_df.iloc[-1, :].values

    print("Price: "+str(reg.predict(test.reshape(1, -1))[0]*area))
input_pred()

Loai bat dong san : 

                            1. biệt thự, liền kề  

                            2. nhà mặt phố

                            3. căn hộ, chung cư

                            4. hẻm
 1
Dien tich(m2): 60
So tang: 2
Quận/huyện/thị xã:

                        1. hà đông

                        2. thanh xuân

                        3. hoàng mai

                        4. cầu giấy

                        5. đống đa

                        6. hai bà trưng
5
Phường/xã/thị trấn: {'nam đồng', 'văn miếu', 'láng thượng', 'trung phụng', 'kim liên', 'cát linh', 'láng hạ', 'ô chợ dừa', 'hàng bột', 'quang trung', 'trung tự', 'trung liệt', 'quốc tử giám', 'khâm thiên', 'thổ quan', 'văn chương', 'phương mai', 'khương thượng', 'ngã tư sở', 'phương liên', 'thịnh quang'}
văn miếu
Price: 8225.043640136719


### Xgboost

In [70]:
# model - getdummy and not normalize 
model1 = XGBRegressor()
t1 = time.time()
model1.fit(X_train, y_train, eval_metric='rmse', verbose=True)
t2 = time.time()
predictions1 = model1.predict(X_test)
rmse1 = np.sqrt(mean_squared_error(y_test, predictions1))
print('rmse = ', rmse1)
mae = mean_absolute_error(y_test, predictions1)
print('mae = ', mae)
print('mse = ', mean_squared_error(y_test, predictions1))
print('time to train: ', (t2-t1))

[07:43:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse =  49.18455521931704
mae =  29.661544769750762
mse =  2419.120472122047
time to train:  18.075910329818726


In [ ]:
print(model1)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


In [ ]:
dump(model1, "../../Code/model.joblib.dat")
print("Saved model to: model.joblib.dat")

Saved model to: model.joblib.dat


In [74]:
xgb = load("../../Code/model.joblib.dat")

[07:45:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [72]:
from xgboost import XGBRegressor

In [75]:
def input_pred():
    house = np.ones(data_train.shape[1])
    type_house = int(input("""Loai bat dong san : \n
                            1. biệt thự, liền kề  \n
                            2. nhà mặt phố\n
                            3. căn hộ, chung cư\n
                            4. hẻm\n """))
    area = float(input("Dien tich(m2): "))
    floor = int(input("So tang: "))
    district = int(input("""Quận/huyện/thị xã:\n
                        1. hà đông\n
                        2. thanh xuân\n
                        3. hoàng mai\n
                        4. cầu giấy\n
                        5. đống đa\n
                        6. hai bà trưng\n"""))
    district_name = None
    for dis in dict_district.keys():
        if dict_district[dis] == district:
            district_name = dis
            break
    ward = unicodedata.normalize("NFD", input("Phường/xã/thị trấn: "+str(address[district_name]['ward'])+"\n").strip())

    ward_district = dict_ward[ward+", "+district_name]

    data_test1 = data_test.copy()
    df = pd.DataFrame([[type_house, area, district, ward_district, floor]], columns = ['type','area','En_district', 'ward, district', 'floor'])
    data_test_df = data_test1.iloc[:, :-1].append(df, ignore_index=True)

    en_data_test_df = pd.get_dummies(data_test_df, columns=['type','En_district','ward, district'])
    test = en_data_test_df.iloc[-1, :].values
    print("Price: "+str(xgb.predict(test.reshape(1, -1))[0]*area))
input_pred()

Loai bat dong san : 

                            1. biệt thự, liền kề  

                            2. nhà mặt phố

                            3. căn hộ, chung cư

                            4. hẻm
 1
Dien tich(m2): 60
So tang: 2
Quận/huyện/thị xã:

                        1. hà đông

                        2. thanh xuân

                        3. hoàng mai

                        4. cầu giấy

                        5. đống đa

                        6. hai bà trưng
5
Phường/xã/thị trấn: {'nam đồng', 'văn miếu', 'láng thượng', 'trung phụng', 'kim liên', 'cát linh', 'láng hạ', 'ô chợ dừa', 'hàng bột', 'quang trung', 'trung tự', 'trung liệt', 'quốc tử giám', 'khâm thiên', 'thổ quan', 'văn chương', 'phương mai', 'khương thượng', 'ngã tư sở', 'phương liên', 'thịnh quang'}
văn miếu
Price: 7744.954833984375
